In [1]:
import pandas as pd
import numpy as np
import os
import nltk

In [2]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

131072

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# this is your data file, however you saved it!
words_file = '/Users/vassiki/Desktop/narrative_complexity/data/scriptdata_final.csv'

words_df = pd.read_csv(words_file)

Reading in just one script in order to prototype the code

In [5]:
script_one = words_df.script[0]
tokens = nltk.word_tokenize(script_one)
cleaned_tokens = [t.lower() for t in tokens if len(t) > 2]

Sentiment analysis with nltk

In [6]:
def sentiment_extractor(word_tokens):
    df = pd.DataFrame()
    for w in word_tokens:
        sid = SentimentIntensityAnalyzer()
        ss = sid.polarity_scores(w)
        res = pd.DataFrame.from_records([ss])
        res['word'] = w
        df = pd.concat([df, res], ignore_index=False)
    return df

In [7]:
# this cell will take a while to execute!
sentiments = sentiment_extractor(cleaned_tokens)

A little toy example for validation purposes

In [8]:
listwords = ['happy', 'sad', 'angry', 'terrible', 'perfect', 'positive', 'negative', 'evil', 'patriarchy', 'love', 'guns', 'trauma', 'war', 'trump']
toy_sentiments = sentiment_extractor(listwords)
toy_sentiments

,neg,neu,pos,compound,word
0,0.0,0.0,1.0,0.5719,happy
0,1.0,0.0,0.0,-0.4767,sad
0,1.0,0.0,0.0,-0.5106,angry
0,1.0,0.0,0.0,-0.4767,terrible
0,0.0,0.0,1.0,0.5719,perfect
0,0.0,0.0,1.0,0.5574,positive
0,1.0,0.0,0.0,-0.5719,negative
0,1.0,0.0,0.0,-0.6597,evil
0,0.0,1.0,0.0,0.0000,patriarchy
0,0.0,0.0,1.0,0.6369,love


In [ ]:
# crude visualization
x = np.arange(sentiments.shape[0])
y = sentiments.sentiment_compound
plt.plot(x, y)

Obtaining word embeddings for all words in a window, one at a time

In [9]:
glove_embeddings = '/Users/vassiki/Downloads/glove.6B/glove.6B.50d.txt'
# you can download these from https://nlp.stanford.edu/projects/glove/
# i am using the smallest version of the pretrained models in 50 dimensions

In [10]:
def load_embeddings(embedding_file):
    emb = {}
    with open(embedding_file, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            emb[''.join(row[0])] = ' '.join(row[1:])
    return emb

In [11]:
# this might take a couple of minutes!
embedding = load_embeddings(glove_embeddings)

In [14]:
def embedding_extractor(tokens, embedding):
    weights = []
    for tok in cleaned_tokens:
        if tok in embedding.keys():
            weights.append(np.array([float(val) for val in embedding[tok].split(' ')]))
        else:
            weights.append(np.zeros(50))
    return pd.DataFrame({'words' : tokens, 'weights' : weights})

In [15]:
emb_df = embedding_extractor(cleaned_tokens, embedding)